In [ ]:
import time
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dense, Dropout, Activation,Bidirectional 
from sklearn.metrics import f1_score, classification_report
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

#List all available activation functions in tf.keras.activations
activation_functions = dir(tf.keras.activations)
# Filter out the built-in attributes
activation_functions = [func for func in activation_functions if not func.startswith('__')]
print(activation_functions)

# Define the MishRelu activation function
def MishRelU(x):
    return tf.where(x > 0, x, x * tf.keras.activations.tanh(tf.keras.activations.softplus(x)))
get_custom_objects()['MishRelU'] = MishRelU
# Example
x = tf.convert_to_tensor([-5,2,4], dtype=tf.float32)
MishRelU(x)

#Define ReLU activation function
def ReLU(x):
    return tf.keras.activations.relu(x)
get_custom_objects()['ReLU'] = ReLU
# Example
x = tf.convert_to_tensor([-5,2,4], dtype=tf.float32)
ReLU(x)

#Define Mish activation function
def Mish(x):
    return x * tf.keras.activations.tanh(tf.keras.activations.softplus(x))
get_custom_objects()['Mish'] = Mish
# Example
x = tf.convert_to_tensor([-5,2,4], dtype=tf.float32)
Mish(x)

#Define Elu activation function
def Elu(x,a=1):
    return  tf.keras.activations.elu(x, alpha=a)    
get_custom_objects()['Elu'] = Elu
# Example
x = tf.convert_to_tensor([-5,2,4], dtype=tf.float32)
Elu(x)

#Define LeakyReLU activation function
def LeakyReLU(x):
    return tf.keras.layers.LeakyReLU(alpha=0.01)(x)
get_custom_objects()['LeakyReLU'] = LeakyReLU
# Example
x = tf.convert_to_tensor([-5,2,4], dtype=tf.float32)
LeakyReLU(x)

#Define Softplus activation function
def Softplus(x):
    return tf.keras.activations.softplus(x)
get_custom_objects()['Softplus'] = Softplus
# Example
x = tf.convert_to_tensor([-5,2,4], dtype=tf.float32)
Softplus(x)

#Define Tanh activation function
def Tanh(x):
    return tf.keras.activations.tanh(x)
get_custom_objects()['Tanh'] = Tanh
# Example
x = tf.convert_to_tensor([-5,2,4], dtype=tf.float32)
Tanh(x)

#Define Selu activation function
def Selu(x):
    return tf.keras.activations.selu(x)
get_custom_objects()['Selu'] = Selu
# Example
x = tf.convert_to_tensor([-5,2,4], dtype=tf.float32)
Selu(x)

#BUILD CNN MODEL 
def build_mnist_model_cnn3(activation, optimizer_name='adam', learning_rate=0.001):
    model = Sequential()
    model.add(Input(shape=(32, 32, 3)))
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(layers.Activation(activation))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Dropout(0.25))

    model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same'))
    model.add(BatchNormalization())
    model.add(layers.Activation(activation))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Dropout(0.25))

    model.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding='Same'))
    model.add(BatchNormalization())
    model.add(layers.Activation(activation))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(layers.Dense(512))
    model.add(BatchNormalization())
    model.add(layers.Activation(activation))
    model.add(Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))

    if optimizer_name.lower() == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

#Build MLP MODEL USEING FASHION-MNIST DATASET
def build_fashion_model(activation, optimizer_name='adam', learning_rate=0.001):
    # Sequential 
    model = keras.Sequential()
    model.add(layers.Input(shape=(28, 28)))
    model.add(layers.Flatten())
    model.add(layers.Dense(units=398))
    model.add(BatchNormalization())
    model.add(layers.Activation(activation))
    model.add(layers.Dropout(rate=0.25)) 
    
    model.add(layers.Dense(units=128))
    model.add(BatchNormalization())
    model.add(layers.Activation(activation))
    model.add(layers.Dropout(rate=0.25))
    
    model.add(layers.Dense(units=64))
    model.add(BatchNormalization())
    model.add(layers.Activation(activation))
    model.add(layers.Dropout(rate=0.25))
    # Output layer
    model.add(layers.Dense(units=10, activation='softmax'))
    #print(model.summary())
    # Select optimizer 
    if optimizer_name.lower() == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# LSTM MODEL BUILDING
def build_lstm_model(activation, learning_rate):
    model = Sequential()
    model.add(Embedding(input_dim =5000, output_dim = 128, input_length = 200))
    model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(32, kernel_regularizer=regularizers.l2(0.001)))
    model.add(Activation(activation))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    # Use Adam optimizer 
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model
    
# LSTM MODEL BUILDING
def BiLSTM_model(activation, learning_rate):
    model = Sequential()
    model.add(Embedding(input_dim =10000, output_dim =32,input_length=200))
    model.add(Bidirectional(LSTM(32, return_sequences=False)))
    model.add(Dropout(0.4))
    model.add(Dense(32, kernel_regularizer=regularizers.l2(0.001)))
    model.add(Activation(activation))
    model.add(Dropout(0.5))
    model.add(Dense(46, activation='softmax')) #num_classes=
    # Use Adam optimizer 
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

lr_schedule = ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.2, min_lr=1e-5)

activation_functions = [MishRelU, RelU, Mish, LeakyReLU]
# activation_functions = [Tanh, Softplus, Sigmoid, Selu]
learning_rates = [0.1, 0.01, 0.001, 0.0001]
optimizers = ['Adam']
METRIC_ACCURACY = 'accuracy'
base_log_dir = "logs_relu/gridsearch"

for activation in activation_functions:
    act_name = activation.__name__
    
    for optimizer_name in optimizers:
        for learning_rate in learning_rates:
            start = time.time()
            log_dir = f"{base_log_dir}/{act_name}_{optimizer_name}_lr{learning_rate}"
            writer = tf.summary.create_file_writer(log_dir)
            tensorboard = TensorBoard(
                log_dir=log_dir,
                histogram_freq=1,
                write_graph=True,
                write_images=False
            )
            print(f"Training with {act_name}, {optimizer_name}, lr={learning_rate}")
            # Build model
            model = build_mnist_model_cnn3(
                activation,
                optimizer_name=optimizer_name,
                learning_rate=learning_rate
            )
            callbacks = [tensorboard, lr_schedule]
            
            # Train model
            history = model.fit(
                X_train, y_train,
                epochs=30,
                validation_data=(X_val, y_val),
                batch_size=128,
                callbacks=callbacks
            )

            # Evaluation
            test_loss, test_acc = model.evaluate(X_test, y_test)

            # Predictions
            y_pred_probs = model.predict(X_test)
            y_pred = np.argmax(y_pred_probs, axis=1)
            y_true = np.argmax(y_test, axis=1)

            # F1 Score
            f1 = f1_score(y_true, y_pred, average='macro')
            print("F1-score (macro):", f1)
            print(classification_report(y_true, y_pred))

            # Timing
            end = time.time()
            total_minutes = (end - start) / 60
            print(f"Test Accuracy: {test_acc:.4f} | Time taken: {total_minutes:.2f} min for {act_name}, optimizer={optimizer_name}, learning_rate={learning_rate}")

            # Save model
            model_path = f"{log_dir}/model.keras"
            model.save(model_path)

            # Model size
            size_bytes = os.path.getsize(model_path)
            size_mb = size_bytes / (1024 * 1024)
            print(f"Model size (activation={act_name}): {size_mb:.2f} MB")

print('Training complete!')

    